In [33]:
import random

import numpy as np

def loadDataSet():
    '''实验样本'''
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                   ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                   ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                   ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                   ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                   ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]
    return postingList, classVec
    

def createVocabList(dataSet):
    '''
    获取所有词汇集合
    '''
    vocabSet = set()
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)


def setOfWords2Vec(vocabList, inputSet):
    '''
    将词条向量化
    向量维度等于词汇表长度
    '''
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec


def trainNB0(trainMatrix, trainCategory):
    '''训练算法：求条件概率矩阵
    Parameters:
    trainMatrix - 训练文档矩阵，即setOfWords2Vec返回的returnVec构成的矩阵
    trainCategory - 训练类别标签向量，即loadDataSet返回的classVec
    Returns:
    p0Vect - 非侮辱类的条件概率数组
    p1Vect - 侮辱类的条件概率数组
    pAbusive - 文档属于侮辱类的概率
    '''
    numTrainDocs = len(trainMatrix)  # 训练样本数量
    numWords = len(trainMatrix[0])  # 词汇表长度
    pAbusive = sum(trainCategory) / float(numTrainDocs)  # p(c1) 是侮辱性留言的概率
    #     p0Num = numpy.zeros(numWords); p1Num = numpy.zeros(numWords)
    #     p0Denom = 0.0; p1Denom = 0.0  # 分母值 各自类别的词条数
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)  # 将所有词出现的次数初始化为1
    p0Denom = 2.0
    p1Denom = 2.0  # 分母值 各自类别的词条数
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]  # 统计词频
            p1Denom += sum(trainMatrix[i])  # 总数累加
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    # p(wi|ci) 使用log是为了防止下溢出
    p1Vect = np.log(p1Num / p1Denom)
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive


def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    '''
    Parameters:
    vec2Classify - 待分类的词条数组
    p0Vec - 侮辱类的条件概率数组
    p1Vec -非侮辱类的条件概率数组
    pClass1 - 文档属于侮辱类的概率
    Returns:
    0 - 属于非侮辱类
    1 - 属于侮辱类
    '''
    # 因为转成log 所以都是加；log(pClass1)是p（ci） 类似于向量相似度比较
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p2 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p2:
        return 1
    else:
        return 0


def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    # 生成词向量矩阵
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB0(np.array(trainMat), np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))


def bagOfWord2VecMN(vocabList, inputSet):
    '''词袋模型'''
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec

def  textParse(bigString):
    import re
    listOfTokens = re.split('\W',bigString)
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]


def spamTest():
    docList = []; classList = []; fullText = []
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainingSet = [i for i in range(50)]
    testSet = []

    # 分配十个作为测试集
    for i in range(10):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])

    # 训练
    trainMat = []; trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))

    # 错误数
    errorCount = 0

    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            print('error docIndex', docIndex)
            errorCount += 1

    print('the error rate is: ', float(errorCount)/len(testSet))

In [34]:
listOPosts, listClasses = loadDataSet()
myVocabList = createVocabList(listOPosts)

In [35]:
myVocabList

['worthless',
 'problems',
 'take',
 'love',
 'please',
 'stop',
 'quit',
 'ate',
 'not',
 'is',
 'him',
 'park',
 'stupid',
 'posting',
 'flea',
 'dog',
 'buying',
 'has',
 'licks',
 'to',
 'mr',
 'food',
 'maybe',
 'cute',
 'help',
 'my',
 'steak',
 'I',
 'how',
 'garbage',
 'so',
 'dalmation']

In [36]:
setOfWords2Vec(myVocabList, listOPosts[0])

[0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0]

In [37]:
trainMat = []
# 生成词向量矩阵
for postinDoc in listOPosts:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))

In [38]:
trainMat

[[0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  1],
 [1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  1,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [39]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

In [40]:
p0V,p1V,pAb

(array([-3.25809654, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
        -2.56494936, -3.25809654, -2.56494936, -3.25809654, -2.56494936,
        -2.15948425, -3.25809654, -3.25809654, -3.25809654, -2.56494936,
        -2.56494936, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
        -2.56494936, -3.25809654, -3.25809654, -2.56494936, -2.56494936,
        -1.87180218, -2.56494936, -2.56494936, -2.56494936, -3.25809654,
        -2.56494936, -2.56494936]),
 array([-1.94591015, -3.04452244, -2.35137526, -3.04452244, -3.04452244,
        -2.35137526, -2.35137526, -3.04452244, -2.35137526, -3.04452244,
        -2.35137526, -2.35137526, -1.65822808, -2.35137526, -3.04452244,
        -1.94591015, -2.35137526, -3.04452244, -3.04452244, -2.35137526,
        -3.04452244, -2.35137526, -2.35137526, -3.04452244, -3.04452244,
        -3.04452244, -3.04452244, -3.04452244, -3.04452244, -2.35137526,
        -3.04452244, -3.04452244]),
 0.5)

In [41]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1


In [48]:
spamTest()

the error rate is:  0.0
